In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
import seaborn as sns
sns.set(rc={'figure.figsize':(20,15)})
from functions import create_base, add_basket_info, add_coupon_info
from functions import get_baskets, get_coupons, get_prediction_index, get_3_files
from functions import split_4_way

In [3]:
baskets, coupons, prediction_index = get_3_files()
baskets = baskets[baskets['customer']<2000]

In [4]:
base = create_base(customers=range(1900,2000))
base['basket'] = base['week'].astype(str) + '_' +base['customer'].astype(str)
base = add_basket_info(base, baskets)
base = add_coupon_info(base, baskets, coupons)

In [18]:
x_train, y_train, x_test, y_test = split_4_way(base, target_col='isBought')

# Trying to create categories

In [ ]:
matrix = base.iloc[0:0+250, 5].values
for i in range(1,base.basket.nunique()):
    start = i*250
    matrix = np.vstack((matrix, base.iloc[i:i+250, 5].values))
    
co_df = pd.DataFrame(matrix)
co_matrix = co_df.T.dot(co_df)
co_matrix[co_matrix > 10] = 1
sns.heatmap(co_matrix, annot=False)